In [ ]:
%load_ext autoreload
%autoreload 2


import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

from hist import Hist

import babar_analysis_tools as bat

import pandas as pd

from analysis_variables import *

import os

In [ ]:
start = time.time()

# At Siena
#topdir = '/mnt/qnap/babar_data/bnv_plambda'

# At Bellis' home
topdir = '/home/bellis/babar_data/bnv_plambda'

# On Bellis' laptop
#topdir = './'

#filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'
filename = f'{topdir}/Background_and_signal_SP_modes_Only_Run_1.parquet'

data = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")


# Producing tables for the skims statistics

In [ ]:
df = bat.read_in_dataset_statistics()
df

In [ ]:
dfspinfo = bat.get_SP_cross_sections_and_labels()

dfspinfo

In [ ]:
mask = dfspinfo['SP Mode']==1235
bbbar_xsec = dfspinfo[mask]['Cross section [nb]'].values[0]

mask = dfspinfo['SP Mode']==1237
bbbar_xsec += dfspinfo[mask]['Cross section [nb]'].values[0]

print(f"The total BBbar cross section is {bbbar_xsec} nb")

In [ ]:
mask = df['Data or MC']=='Data'
df[mask]

In [ ]:
mask = (df['Data or MC']=='Data') & (df['Skim']=='LambdaVeryVeryLoose')
dftmp = df[mask][['Run', 'Luminosity (Data only) 1/pb','# of events (Data or MC)', '# of events (Data or MC) NOT SURE WHICH NUMBER TO USE']]

dftmp['# of BBbar pairs'] = dftmp['Luminosity (Data only) 1/pb']*bbbar_xsec*1000

dftmp

In [ ]:

#dftmp.style.concat(df.agg(['sum']).style)

dftmp['Run'] = dftmp['Run'].astype(int).astype(str)
dftmp.loc['Total'] = dftmp.sum(numeric_only=True)

#dftmp.iloc['Total']['Run'] = 'Total'

dftmp

In [ ]:
dftmp.at['Total','Run'] = 'Total'

In [ ]:
header = []
header.append('Run')
header.append('Luminosity (1/pb)')
header.append('\# skimmed events')
header.append('\# org. events')
header.append('\# BB pairs')

caption = "Details of the numbers of events and luminosity from the {\\tt LambdaVeryVeryLoose} skim used in this analysis."
label = 'tab:dataskims'

df.style.to_latex(position_float='centering')

output = dftmp.to_latex(index=False, header=header, float_format="%.1f", caption=caption, label=label)

# Add in centering by replacing the first EOL with "EOL + \centering + EOL"
output = output.replace('\n','\n\centering\n', 1)

# Add an hline learn the bottom above the total
output = output.replace('Total','\hline\nTotal', 1)



#styler = dftmp.style
#output = styler.to_latex(caption=caption, label=label)

print(output)

# Write it out
current_dir= os.getcwd()
print(f"Writing to {current_dir}")
directory = "tables"
path= os.path.join(current_dir,directory)
if os.path.isdir(path)== False:
    os.mkdir(path)

print(f"Writing to {directory}")

outfilename = current_dir+"/"+directory+"/table_data_skim_statistics.tex"
outfile = open(outfilename,'w+')
outfile.write(output)
outfile.close()
